In [1]:
import pandas as pd
import ee
import pandas as pd
from prophet import Prophet
import streamlit as st
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import folium

In [2]:
ee.Initialize()

In [3]:
aoi = ee.Geometry.Polygon(
    [[[126.19653266055587, 36.853168924185226],
      [126.19653266055587, 36.83836240145186],
      [126.21259510596076, 36.83836240145186],
      [126.21259510596076, 36.853168924185226],
      [126.19653266055587, 36.853168924185226]]]
)

In [4]:
start_date = '2021-01-01'
end_date = '2023-10-31'

In [5]:
def calculateRVI(aoi,start_date,end_date):
    # Sentinel-1 ImageCollection 필터링
    sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
            .filterBounds(aoi) \
            .filterDate(start_date, end_date) \
            .filter(ee.Filter.eq('instrumentMode', 'IW')) \
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
            .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    # RVI 계산 및 시계열 데이터 생성 함수
    def calculate_rvi(image):
        date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
        vv = image.select('VV')
        vh = image.select('VH')
        rvi = vh.multiply(4).divide(vv.add(vh)).rename('rvi')
        mean_rvi = rvi.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=aoi,
            scale=10
        ).get('rvi')
        return ee.Feature(None, {'ds': date, 'y': mean_rvi})
    # 시계열 RVI 계산
    time_series_rvi = sentinel1.map(calculate_rvi)
    # 결과를 서버측 객체로 변환 (Python 클라이언트로 가져오기 위함)
    rvi_features = time_series_rvi.getInfo()['features']
    # 결과를 pandas DataFrame으로 변환
    df = pd.DataFrame([feat['properties'] for feat in rvi_features])
    # DataFrame을 'Date' 컬럼에 따라 오름차순으로 정렬
    df = df.sort_values(by='ds')
    return df

In [6]:
def prophet_process(df):
    # Prophet
    # m = Prophet(yearly_seasonality=True,daily_seasonality=False,weekly_seasonality=False,holidays_prior_scale=0,changepoint_prior_scale=0.5)
    m = Prophet(yearly_seasonality=True,daily_seasonality=False,weekly_seasonality=False,holidays_prior_scale=0.01,changepoint_prior_scale=0.01)
    m.fit(df)
    # future dataframe 생성
    future = m.make_future_dataframe(periods=0,freq='M')
    # predict
    forecast = m.predict(future) 
    forecasted_value = forecast.iloc[-1]['yhat']  # 예측된 값을 가져옴
    # 예측 결과를 데이터프레임에 추가
    new_row = pd.DataFrame({'ds': [future.iloc[-1]['ds']], 'y': [forecasted_value]})
    forecast_df = pd.concat([df, new_row], ignore_index=True)
    return forecast,forecast_df,df,m

In [7]:
def ts_analysis(df):
    # 날짜 컬럼을 datetime 형식으로 변환
    df['ds'] = pd.to_datetime(df['ds'])

    # 최대값과 최소값의 일자 찾기
    max_date = df[df['yhat'] == df['yhat'].max()]['ds'].iloc[0]
    min_date = df[df['yhat'] == df['yhat'].min()]['ds'].iloc[0]

    # 계절을 구분하기 위한 함수 정의
    def get_season(month):
        if month in [3, 4, 5]:
            return 'Spring'
        elif month in [6, 7, 8]:
            return 'Summer'
        elif month in [9, 10, 11]:
            return 'Fall'
        else:
            return 'Winter'

    # 월별 계절 할당
    df['season'] = df['ds'].dt.month.apply(get_season)

    # 계절별 평균값 계산
    seasonal_trend = df.groupby('season')['yhat'].mean()

    # 매월 평균 계산
    monthly_avg = df.groupby(df['ds'].dt.month)['yhat'].mean()

    # 전체 기간에 대한 평균 yhat 값
    overall_avg = df['yhat'].mean()

    # 계절별 평균값을 전체 평균값으로 나누어 상대적인 비율 계산
    seasonal_relative = seasonal_trend / overall_avg

    # 매년 평균값 계산
    annual_avg = df.groupby(df['ds'].dt.year)['yhat'].mean()

    # 매년 평균값을 전체 평균값으로 나누어 상대적인 비율 계산
    annual_relative = annual_avg / overall_avg

    # 매월 평균값을 전체 평균값으로 나누어 상대적인 비율 계산
    monthly_relative = monthly_avg / overall_avg

    result_df = pd.DataFrame({'seasonal_relative': seasonal_relative,
                              'annual_relative': annual_relative,
                              'monthly_relative': monthly_relative
    })

    return result_df, max_date, min_date, seasonal_trend


In [8]:
df_s = calculateRVI(aoi,start_date,end_date)

In [9]:
forecast,forecast_df,df,m = prophet_process(df_s)

16:56:14 - cmdstanpy - INFO - Chain [1] start processing
16:56:14 - cmdstanpy - INFO - Chain [1] done processing


In [10]:
result_df, max_date, min_date, seasonal_trend = ts_analysis(forecast)

In [11]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler # 출력을 스트리밍하는 데 사용
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent


def process_llm(df):
    template = """
    You are an analyst who explains the vegetation index. 
    {question} has a relative ratio of four seasons.
    Please explain this to me.
    """

    prompt = PromptTemplate(template=template, input_variables=["question"])

    callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

    llm = LlamaCpp(
                    model_path="firefly-llama2-13b-chat.Q4_K_M.gguf",
                    input={"temperature": 0.75,
                        "max_length": 4000,
                        "top_p": 1},
                    callback_manager=callback_manager,
                    n_ctx=2048
                    )

    agent = create_pandas_dataframe_agent(llm, df, verbose=True)

    answer = agent.run(prompt)

    return answer 